In [ ]:
import json
from pprint import pprint
import os

import pandas as pd

from kf_model_fhir.client import FhirApiClient

In [ ]:
FHIR_USER = os.environ('FHIR_USER')
FHIR_PASS = os.environ('FHIR_PASS')
BASE_URL = 'http://10.10.1.191:8000'
FHIR_VERSION = '4.0.0'

client = FhirApiClient(
    base_url=BASE_URL, 
    fhir_version=FHIR_VERSION, 
    auth=(FHIR_USER, FHIR_PASS)
)

In [ ]:
def validate(
        payload,
        expected_success=True, 
        print_result=False,
    ):
    rt = payload.get('resourceType')
    endpoint = f'{client.base_url}/{rt}'
    
    success = client.delete_all(
        endpoint, params={'url': payload['url']}
    )
    print(f'DELETE {payload["url"]}')
    assert success
    
    success, result = client.send_request(
        'POST', endpoint, json=payload
    )
    
    if print_result:
        pprint(result)
        
    print(f'POST {endpoint}, {payload.get("name")}')
    print(success == expected_success)    

# Mondo

In [ ]:
mondo_cs = {
    "resourceType": "CodeSystem",
    "id": "disease-term-mondo",
    "url": "http://fhir.kids-first.io/CodeSystem/disease-term-mondo",
    "version": "0.1.0",
    "name": "DiseaseTerminologyMondo",
    "title": "Disease Terminology Mondo",
    "status": "active",
    "experimental": True,
    "publisher": "Kids First DRC",
    "description": "Imported Mondo terminology from OBO PURLs.",
    "caseSensitive": True,
    "valueSet": "http://fhir.kids-first.io/ValueSet/disease-term",
    "content": "complete",
}

In [ ]:
# With Avi's ontolpogy scraper
cs_mondo = json.load(open('./codesystem_mondo.json'))
mondo_cs['count'] = cs_mondo['count']
mondo_cs['concept'] = cs_mondo['concept']

In [ ]:
# With custom terminology from data
mondo_concepts = []
df = pd.read_csv('./disease_terminology.tsv', sep='\t')
for i, group in df[df.mondo_id_diagnosis != 'No Match'][[
        'source_text_diagnosis', 
        'mondo_id_diagnosis'
    ]].drop_duplicates().groupby('mondo_id_diagnosis'):
    for _, row in group.iterrows():
        mondo_concepts.append({
            'code': row.mondo_id_diagnosis,
            'display': row.source_text_diagnosis
        })
        break

mondo_cs['count'] = len(mondo_concepts)
mondo_cs['concept'] = mondo_concepts

In [ ]:
json.dump(
    mondo_cs, 
    open('./results/CodeSystem-disease-term-mondo.json', 'w'), 
    separators=(',', ':')
)

In [ ]:
validate(mondo_cs)

# NCIt

In [ ]:
ncit_cs = {
    "resourceType": "CodeSystem",
    "id": "disease-term-ncit",
    "url": "http://fhir.kids-first.io/CodeSystem/disease-term-ncit",
    "version": "0.1.0",
    "name": "DiseaseTerminologyNCIt",
    "title": "Disease Terminology NCIt",
    "status": "active",
    "experimental": True,
    "publisher": "Kids First DRC",
    "description": "Imported NCIt terminology from OBO PURLs.",
    "caseSensitive": True,
    "valueSet": "http://fhir.kids-first.io/ValueSet/disease-term",
    "content": "complete"
}

In [ ]:
# With Avi's ontolpogy scraper
cs_ncit = json.load(open('./codesystem_ncit.json'))
ncit_cs['count'] = cs_ncit['count']
ncit_cs['concept'] = cs_ncit['concept']

In [ ]:
# With custom terminology from data
ncit_concepts = []
df = pd.read_csv('./disease_terminology.tsv', sep='\t')
for i, group in df[df.ncit_id_diagnosis != 'No Match'][[
        'source_text_diagnosis', 
        'ncit_id_diagnosis'
    ]].drop_duplicates().groupby('ncit_id_diagnosis'):
    for _, row in group.iterrows():
        ncit_concepts.append({
            'code': row.ncit_id_diagnosis,
            'display': row.source_text_diagnosis
        })
        break
            
ncit_cs['count'] = len(ncit_concepts)
ncit_cs['concept'] = ncit_concepts

In [ ]:
json.dump(
    ncit_cs, 
    open('./results/CodeSystem-disease-term-ncit.json', 'w'), 
    separators=(',', ':')
)

In [ ]:
validate(ncit_cs)

# Value Set

In [ ]:
disease_term_vs = {
    "resourceType": "ValueSet",
    "id": "disease-term",
    "url": "http://fhir.kids-first.io/ValueSet/disease-term",
    "version": "0.1.0",
    "name": "DiseaseTerminology",
    "title": "Disease Terminology",
    "status": "active",
    "experimental": True,
    "publisher": "Kids First DRC",
    "description": "Represents the biomedical terminologies of a disease.",
    "compose": {
        "include": [
            {
                "system": "http://fhir.kids-first.io/CodeSystem/disease-term-mondo"
            },
            {
                "system": "http://fhir.kids-first.io/CodeSystem/disease-term-ncit"
            }
        ]
    }
}

In [ ]:
json.dump(    
    disease_term_vs, 
    open('./results/ValueSet-disease-term.json', 'w'), 
    separators=(',', ':')
)

In [ ]:
validate(disease_term_vs)

# Structure Definition

## Disease Termology Extension

In [ ]:
# Here's what a simple key-value extension with a string data type looks like:
disease_term_ext = {
    "resourceType" : "StructureDefinition",
    "id": "disease-term",
    "extension" : [
        {
            "url" : "http://hl7.org/fhir/StructureDefinition/structuredefinition-wg",
            "valueCode" : "oo"
        },
        {
            "url" : "http://hl7.org/fhir/StructureDefinition/structuredefinition-fmm",
            "valueInteger" : 1
        }
    ],
    "url" : "http://fhir.kids-first.io/StructureDefinition/disease-term",
    "version" : "0.1.0",
    "name" : "DiseaseTerminology",
    "title" : "Disease Terminology",
    "status" : "active",
    "date" : "2020-03-05",
    "publisher" : "Kids First DRC",
    "description" : "Disease terminology.",
    "fhirVersion" : "4.0.0",
    "kind" : "complex-type",
    "abstract" : False,
    "context" : [{
        "type" : "element",
        "expression" : "Condition"
    }],
    "type" : "Extension",
    "baseDefinition" : "http://hl7.org/fhir/StructureDefinition/Extension",
    "derivation" : "constraint",
    "differential" : {
        "element": [
            {
                "id" : "Extension",
                "path" : "Extension",
                "short" : "Disease terminology",
                "definition" : "Disease terminology",
                "min" : 0,
                "max" : "1",
            },
            {
                "id" : "Extension.extension",
                "path" : "Extension.extension",
                "max" : "0"
            },
            {
                "id" : "Extension.url",
                "path" : "Extension.url",
                "type": [
                    {
                        "code": "uri"
                    }
                ],
                "fixedUri" : "http://fhir.kids-first.io/StructureDefinition/disease-term"
            },
            {
                "id": "Extension.valueCodeableConcept",
                "path": "Extension.valueCodeableConcept",
                "type": [
                    {
                        "code": "CodeableConcept"
                    }
                ],
                "binding": {
                    "extension": [
                        {
                            "url": "http://hl7.org/fhir/StructureDefinition/elementdefinition-bindingName",
                            "valueString": "Disease Terminology"
                        }
                    ],
                    "strength": "required",
                    "valueSet": "http://fhir.kids-first.io/ValueSet/disease-term"
                }
            }
        ]
    }
}

In [ ]:
json.dump(
    disease_term_ext, 
    open('./results/StructureDefinition-disease-term.json', 'w'), 
    separators=(',', ':')
)

In [ ]:
validate(disease_term_ext)

## Disease

In [ ]:
disease_sd = json.load(open('../../../site_root/source/resources/StructureDefinition-Disease.json'))

In [ ]:
disease_sd['differential']['element'].insert(3,
    {
        'id': 'Condition.extension:term',
        'path': 'Condition.extension',
        'sliceName': 'term',
        'short': 'Disease terminology',
        'definition': 'Disease terminology.',
        'type': [{
            'code': 'Extension',
            'profile': ['http://fhir.kids-first.io/StructureDefinition/disease-term']
        }],
        'mustSupport': True
    }
)

In [ ]:
json.dump(
    disease_sd, 
    open('./results/StructureDefinition-Disease.json', 'w'), 
    separators=(',', ':')
)

In [ ]:
validate(disease_sd)

# Search Parameter

In [ ]:
disease_term_sp = {
    "resourceType": "SearchParameter",
    "id": "disease-term",
    "url": "http://fhir.kids-first.io/SearchParameter/disease-term",
    "name": "disease-term",
    "status": "active",
    "experimental": True,
    "publisher": "Kids First DRC",
    "description": "Search for Diseases by terminology",
    "code": "term",
    "base": ["Condition"],
    "type": "token",
    "expression": "Condition.extension.where(url = 'http://fhir.kids-first.io/StructureDefinition/disease-term').value",
    "target": ["Condition"]
}

In [ ]:
json.dump(
    disease_term_sp, 
    open('./results/SearchParameter-disease-term.json', 'w'), 
    separators=(',', ':')
)

In [ ]:
validate(disease_term_sp)